# Instalação de bibliotecas

In [ ]:
pip install keras

In [32]:
 pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [33]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [34]:
import pandas as pd
import numpy as np
from keras.preprocessing.text import Tokenizer
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer

Abrindo o dataframe já pré - processado, ou seja, que passou pela limpeza, tokenização e remoção de stop words.

In [35]:
pre_processing = pd.read_csv('../output/pre_processamento.csv')

In [36]:
pre_processing

,dataPublicada,autor,texto,sentimento,tipoInteracao,anomalia,probabilidadeAnomalia,linkPost,processado,contemHyperlink,texto_e_palavra,texto_tokenizado
0,"""2022-03-04 08:36:00""",lmviapiana,Minuto touro de ouro,POSITIVE,comentário,0,38,https://www.instagram.com/p/CapXhB5Lvas/#17977...,0,0,Minuto touro de ouro,"['minuto', 'touro', 'ouro']"
1,"""2022-03-03 21:10:00""",vanilson_dos,@ricktolledo Sim,NEUTRAL,resposta,0,17,https://www.instagram.com/p/CapXhB5Lvas/#17842...,0,0,@ricktolledo Sim,['sim']
2,"""2022-03-03 20:54:00""",ricktolledo,Queria saber se a Btg banking é a própria btg ...,POSITIVE,comentário,0,20,https://www.instagram.com/p/CapXhB5Lvas/#17935...,0,0,Queria saber se a Btg banking é a própria btg ...,"['queria', 'saber', 'banking', 'própria']"
3,"""2022-03-03 18:56:00""",claudiofalavinha,@thaotinhasbfc,NEUTRAL,comentário,0,41,https://www.instagram.com/p/CapXhB5Lvas/#17942...,0,0,@thaotinhasbfc,[]
4,"""2022-03-03 16:11:00""",paularodrigues.invest,👏👏👏👏,NEUTRAL,comentário,0,15,https://www.instagram.com/p/CaptWrtOT8U/#17943...,0,0,palmas palmas palmas palmas,"['palmas', 'palmas', 'palmas', 'palmas']"
...,...,...,...,...,...,...,...,...,...,...,...,...
6351,"""2022-11-30 07:48:00""",perspectiveinvestimentos,Excelente explicação,POSITIVE,comentário,0,15,https://www.instagram.com/p/ClgWR-Ov2H1/#18001...,0,0,Excelente explicação,"['excelente', 'explicação']"
6352,"""2022-11-30 08:42:00""",marlenenelso,Porque o brg tá diminuído o limite do cart...,NEGATIVE,comentário,1,53,https://www.instagram.com/p/ClgWR-Ov2H1/#17878...,0,0,Porque o brg tá diminuído o limite do cart...,"['porque', 'brg', 'tá', 'diminuído', 'limite',..."
6353,"""2022-11-30 11:27:00""",eduardocolares,Atendam o telefone pelo amor de Deus,POSITIVE,comentário,0,28,https://www.instagram.com/p/CllkSBOuKlJ/#17960...,0,0,Atendam o telefone pelo amor de Deus,"['atendam', 'telefone', 'amor', 'deus']"
6354,"""2022-11-30 12:18:00""",jayipacderota,Estou muito grato por todo o esforço que você ...,POSITIVE,comentário,1,52,https://www.instagram.com/p/CllkSBOuKlJ/#17960...,0,0,Estou muito grato por todo o esforço que você ...,"['grato', 'todo', 'esforço', 'rendeu', 'invest..."


In [37]:
# Função para classificar os sentimentos em colunas
def classify_feeling(sentimento):
    if sentimento == 'POSITIVE':
        return 1, 0, 0
    elif sentimento == 'NEGATIVE':
        return 0, 1, 0
    else:
        return 0, 0, 1

In [40]:
valores_classificados = pre_processing['sentimento'].apply(classify_feeling)

In [41]:
# Aplicar a classificação de sentimentos em colunas separadas
pre_processing[['positivo', 'negativo', 'neutro']] = pd.DataFrame(valores_classificados.tolist())

In [42]:
pre_processing.drop('sentimento', axis=1, inplace=True)
pre_processing

,dataPublicada,autor,texto,tipoInteracao,anomalia,probabilidadeAnomalia,linkPost,processado,contemHyperlink,texto_e_palavra,texto_tokenizado,positivo,negativo,neutro
0,"""2022-03-04 08:36:00""",lmviapiana,Minuto touro de ouro,comentário,0,38,https://www.instagram.com/p/CapXhB5Lvas/#17977...,0,0,Minuto touro de ouro,"['minuto', 'touro', 'ouro']",1,0,0
1,"""2022-03-03 21:10:00""",vanilson_dos,@ricktolledo Sim,resposta,0,17,https://www.instagram.com/p/CapXhB5Lvas/#17842...,0,0,@ricktolledo Sim,['sim'],0,0,1
2,"""2022-03-03 20:54:00""",ricktolledo,Queria saber se a Btg banking é a própria btg ...,comentário,0,20,https://www.instagram.com/p/CapXhB5Lvas/#17935...,0,0,Queria saber se a Btg banking é a própria btg ...,"['queria', 'saber', 'banking', 'própria']",1,0,0
3,"""2022-03-03 18:56:00""",claudiofalavinha,@thaotinhasbfc,comentário,0,41,https://www.instagram.com/p/CapXhB5Lvas/#17942...,0,0,@thaotinhasbfc,[],0,0,1
4,"""2022-03-03 16:11:00""",paularodrigues.invest,👏👏👏👏,comentário,0,15,https://www.instagram.com/p/CaptWrtOT8U/#17943...,0,0,palmas palmas palmas palmas,"['palmas', 'palmas', 'palmas', 'palmas']",0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6351,"""2022-11-30 07:48:00""",perspectiveinvestimentos,Excelente explicação,comentário,0,15,https://www.instagram.com/p/ClgWR-Ov2H1/#18001...,0,0,Excelente explicação,"['excelente', 'explicação']",1,0,0
6352,"""2022-11-30 08:42:00""",marlenenelso,Porque o brg tá diminuído o limite do cart...,comentário,1,53,https://www.instagram.com/p/ClgWR-Ov2H1/#17878...,0,0,Porque o brg tá diminuído o limite do cart...,"['porque', 'brg', 'tá', 'diminuído', 'limite',...",0,1,0
6353,"""2022-11-30 11:27:00""",eduardocolares,Atendam o telefone pelo amor de Deus,comentário,0,28,https://www.instagram.com/p/CllkSBOuKlJ/#17960...,0,0,Atendam o telefone pelo amor de Deus,"['atendam', 'telefone', 'amor', 'deus']",1,0,0
6354,"""2022-11-30 12:18:00""",jayipacderota,Estou muito grato por todo o esforço que você ...,comentário,1,52,https://www.instagram.com/p/CllkSBOuKlJ/#17960...,0,0,Estou muito grato por todo o esforço que você ...,"['grato', 'todo', 'esforço', 'rendeu', 'invest...",1,0,0


# Aplicação Bag of Words

Durante esta etapa é feita a implementação do modelo Bag of Words, que vetoriza as palavras e faz a contagem de vezes que cada uma aparece na coluna a ser utilizada. Assim, tendo como resultado um dataframe com as palavras que mais aparecem no texto e sua respectiva frequência.

In [43]:
# def bow_dataframe(inputUser):
#     tokenizer = Tokenizer()
#     tokenizer.fit_on_texts(inputUser)
#     # Com essa variavel é possível ver as palavras e contar sua frequência
#     wordCount = tokenizer.word_counts    
#     dfCountBoW = pd.DataFrame(list(wordCount.items()))
#     dfCountBoW.rename(columns={0: "Palavra", 1:"Frequência"}, inplace=True)
#     final_df = dfCountBoW.sort_values(by=['Frequência'], ascending=False)
#     return final_df

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

def bow(text):

  # create the transform
  vectorizer = CountVectorizer()

  # tokenize and build vocab
  vectorizer.fit(text)

  # summarize
  vocabulario = sorted(vectorizer.vocabulary_)

  # encode document
  vector = vectorizer.transform(text)

  # summarize encoded vector
  return (vocabulario, vector.toarray())

def processar_frases(frases):

  frases_processadas = []

  for frase in frases:
    frase_processada = pipeline(frase)
    frases_processadas.append(frase_processada)

  return frases_processadas

In [ ]:
# Testando a função 
bow_test = pre_processing['texto_tokenizado']
bow_output = bow_test.apply(bow)
bow_output

#### Caso de teste

In [44]:
# Testando a função 
# bow_test = pre_processing
# bow_test_result = bow_dataframe(bow_test['texto_tokenizado'])

In [45]:
# Input da função
# pre_processing

,dataPublicada,autor,texto,tipoInteracao,anomalia,probabilidadeAnomalia,linkPost,processado,contemHyperlink,texto_e_palavra,texto_tokenizado,positivo,negativo,neutro
0,"""2022-03-04 08:36:00""",lmviapiana,Minuto touro de ouro,comentário,0,38,https://www.instagram.com/p/CapXhB5Lvas/#17977...,0,0,Minuto touro de ouro,"['minuto', 'touro', 'ouro']",1,0,0
1,"""2022-03-03 21:10:00""",vanilson_dos,@ricktolledo Sim,resposta,0,17,https://www.instagram.com/p/CapXhB5Lvas/#17842...,0,0,@ricktolledo Sim,['sim'],0,0,1
2,"""2022-03-03 20:54:00""",ricktolledo,Queria saber se a Btg banking é a própria btg ...,comentário,0,20,https://www.instagram.com/p/CapXhB5Lvas/#17935...,0,0,Queria saber se a Btg banking é a própria btg ...,"['queria', 'saber', 'banking', 'própria']",1,0,0
3,"""2022-03-03 18:56:00""",claudiofalavinha,@thaotinhasbfc,comentário,0,41,https://www.instagram.com/p/CapXhB5Lvas/#17942...,0,0,@thaotinhasbfc,[],0,0,1
4,"""2022-03-03 16:11:00""",paularodrigues.invest,👏👏👏👏,comentário,0,15,https://www.instagram.com/p/CaptWrtOT8U/#17943...,0,0,palmas palmas palmas palmas,"['palmas', 'palmas', 'palmas', 'palmas']",0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6351,"""2022-11-30 07:48:00""",perspectiveinvestimentos,Excelente explicação,comentário,0,15,https://www.instagram.com/p/ClgWR-Ov2H1/#18001...,0,0,Excelente explicação,"['excelente', 'explicação']",1,0,0
6352,"""2022-11-30 08:42:00""",marlenenelso,Porque o brg tá diminuído o limite do cart...,comentário,1,53,https://www.instagram.com/p/ClgWR-Ov2H1/#17878...,0,0,Porque o brg tá diminuído o limite do cart...,"['porque', 'brg', 'tá', 'diminuído', 'limite',...",0,1,0
6353,"""2022-11-30 11:27:00""",eduardocolares,Atendam o telefone pelo amor de Deus,comentário,0,28,https://www.instagram.com/p/CllkSBOuKlJ/#17960...,0,0,Atendam o telefone pelo amor de Deus,"['atendam', 'telefone', 'amor', 'deus']",1,0,0
6354,"""2022-11-30 12:18:00""",jayipacderota,Estou muito grato por todo o esforço que você ...,comentário,1,52,https://www.instagram.com/p/CllkSBOuKlJ/#17960...,0,0,Estou muito grato por todo o esforço que você ...,"['grato', 'todo', 'esforço', 'rendeu', 'invest...",1,0,0


In [46]:
# Output da função
# bow_test_result.head(5)

,Palavra,Frequência
8,'palmas',2668
98,'banco',475
193,'muitoengraçado',443
613,'limite',363
16,'conta',319


#### Demonstração da função

Nesta etapa será demonstrado o funcionamento da função na base de dados real 

In [55]:
# result_output = bow_dataframe(pre_processing['texto_tokenizado'])
# result_output.head(10)

,Palavra,Frequência
8,'palmas',2668
98,'banco',475
193,'muitoengraçado',443
613,'limite',363
16,'conta',319
309,'cartão',310
65,'foguete',295
64,'azul',258
63,'coração',239
194,'melhor',228


# Naive bayes

In [56]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import BernoulliNB

# Dados de treinamento
rotulos_treino = pre_processing[['positivo', 'negativo', 'neutro']]
rotulos_treino = rotulos_treino.values

# Converter as frases em uma representação BoW
vectorizer = CountVectorizer(binary=True)
X_train = vectorizer.fit_transform(pre_processing['texto_tokenizado']).toarray()
y_train = rotulos_treino

# Treinar o classificador Naive Bayes com distribuição de Bernoulli
classifier = BernoulliNB()
classifier.fit(X_train, y_train)

# Fazer previsões para as frases de teste
X_test = vectorizer.transform(pre_processing[texto_tokenizado]).toarray()
y_pred = classifier.predict(X_test)

# Exibir as previsões
for frase, predicao in zip(pre_processing['texto_tokenizado'], y_pred):
    print(f"Frases: {frase}")
    print(f"Predição: {predicao}")
    print()


ValueError: y should be a 1d array, got an array of shape (6356, 3) instead.

In [13]:
result_output.to_csv('..\\output\\bow.csv',encoding='utf-8', index=False, header=True, )